# M2.4 - Traitement des enregistrements de données sur le climat long simultanément

*Partie de:* [** Computational Climate Science **](https://github.com/openClimatescience/M2-Compuational-climate-science) | **Leçon précédente** | **Next Leçon**

**Contenu:**

- [Limitations des ressources dans l'informatique](#Resource-Limitations-in-Computing)
  - [Problèmes liés au CPU](#CPU-Bound-Problems)
- [Traitement simultané pour les grands ensembles de données climatiques](#Current-Processing-for-Lor-Climate-Datasets)
- [Computing animal à l'aide de l'équation de Hargreaves](#Computing-Pet-Using-Hargreaves-Equation)
  - [Radiation informatique en informatique (TOA)](#Computing-top-of-atmosphère-(TOA)-radiation)
  - [fonctions bien documentées](#fonctions-bien-documentées)
  - [Fonctions vectorisées](#Vectorized-Functions)
  - [dériver des variables des coordonnées `xarray`](#dériver-variables-from-xarray-coordonnées)
- [Application d'une fonction à des morceaux indépendants](#Application-a-Function-to-Independent-Chunks)
  - [Mappage d'une fonction arbitraire](#mapping-an-arbitrary-fonction)
- [Profilage des ressources informatiques](#Profile-Compuputal-Resources)
  - [Mesurer le temps du mur d'une tâche avec «TimeIt»](#Mesurer-the-wall-time-of-a-task-withitit)
- [Résumé](#Résumé)

## Aperçu

Dans la leçon précédente, nous avons discuté de la façon dont un modèle simple peut être utilisé pour quantifier la différence entre l'approvisionnement en eau (précipitation) et la perte d'eau (évapotranspiration potentielle ou TEP). Le rapport de ces deux quantités est également utile comme indice de la quantité de perte d'eau reconstituée par les précipitations:
$$
\text {Pourcentage reconstitué} \approx  100 \times \frac {\text {Précipitation}} {\text {PET}}
$$

**La méthode de calcul de PET que nous utiliserons est [la méthode Hargreaves](https://www.fao.org/4/x0490e/x0490e07.htm#minimum%20data%20Requirements) (Allen et al. 2000), car cela ne nécessite que des données de température.** Nous utiliserons les données de température de Merra-2 pour calculer le TEP. Ensuite, nous utiliserons les données de précipitations de CHIRPS, encore une fois, pour dériver notre indice de sécheresse hydrologique.

**Bien qu'il existe de nombreuses sources de données PET, nous allons calculer les PET par nous-mêmes afin que nous puissions acquérir plus d'expérience en travaillant avec de grands ensembles de données climatiques.** En cours de route, nous apprendrons à quel point traité **simultanément,** qui peut aider à résoudre deux problèmes communs:

1. L'ensemble de données entier est trop grand pour se charger en mémoire en une seule fois;
2. Le traitement des données peut prendre du temps, soit parce que l'ensemble de données est si grand ou parce que les calculs sont complexes.

---

## Limitations des ressources dans l'informatique

Généralement, plus l'ensemble de données est important, plus les ressources de calcul sont nécessaires pour l'analyser. Mais exactement les ressources nécessaires dépend à la fois des données et du type d'analyse que nous voulons effectuer.

**Dans les problèmes informatiques, il existe trois types principaux de limitations de ressources ou *goulots d'étranglement,* c'est-à-dire des facteurs limitant la gestion d'un algorithme informatique:**

1. **Lire et écrire la vitesse à partir d'un système de fichiers**
2. **Mémoire informatique**
3. **Unité de traitement central (CPU) vitesse d'horloge (par exemple, 3 GHz)**
   
Un goulot d'étranglement de **Type 1** se produit lorsque nous avons des ensembles de données très grands ou des vitesses de lecture de système de fichier lent. La vitesse de lecture et d'écriture à partir d'un système de fichiers (ou disque dur) dépend du support; Les entraînements à l'état solide sont généralement plus rapides que les disques du disque de rotation. Si le lecteur est un périphérique de stockage (NAS) connecté au réseau au lieu de la conduite dure sur votre ordinateur, la vitesse de la connexion réseau fait également partie des goulots d'étranglement de type 1. **Les problèmes limités par un goulot d'étranglement de type 1 sont appelés liés aux E / O (entrée / sortie liés).**

Un goulot d'étranglement de **Type 2** peut se produire si l'ensemble de données est très grand et que nous essayons de tout stocker en mémoire à la fois, ou si notre analyse génère trop de données en mémoire. Bien sûr, la mémoire est finie, donc les données correspondent à la mémoire ou ce n'est pas le cas. Si notre programme informatique est très sophistiqué, il peut décharger certaines données stockées en mémoire sur le disque dur de l'ordinateur. C'est ce qu'on appelle *l'échange* et c'est extrêmement lent. Par conséquent, si vous manquez de mémoire de l'ordinateur, votre programme peut ne pas s'arrêter en raison d'un manque de mémoire, mais cela ralentira gravement car il essaie de jongler des données entre la mémoire et le disque dur. **Les problèmes limités par un goulot d'étranglement de type 2 sont appelés liés à la mémoire.**

Un goulot d'étranglement de **Type 3** a beaucoup moins à voir avec les données et plus à voir avec l'algorithme que nous exécutons. Si nous lisons dans un énorme ensemble de données et faisons simplement une simple conversion d'unité (par exemple, la multiplication des données par 1000, puis les enregistrer sur le disque), alors la vitesse d'horloge du processeur n'est probablement pas un problème: les ordinateurs peuvent multiplier les nombres très rapide. Mais exactement à quelle vitesse dépend de la vitesse à laquelle le processeur est. **Les problèmes limités par un goulot d'étranglement de type 3 sont appelés liés au processeur.**

### Problèmes liés au processeur

Historiquement, les goulots d'étranglement de type 3 ont reçu le plus d'attention. Les améliorations du processus de fabrication des CPU ont conduit à des puces de plus en plus rapides. Gordon Moore a été l'un des premiers à remarquer le taux de cette tendance à la hausse, et ** Law's Law ** est un article de foi dans l'industrie depuis longtemps: la tendance des vitesses de l'horloge CPU à doubler tous les 2 ans (Moore (Moore 1965).

[Mais il y a des signes récents que ce taux de doublement peut ralentir.](Https://www.tomshardware.com/tech-industry/semiconductors/intels-ceo-says-moores-law-is-slowing-to-A-Three-Year-Cadence-mais-It-Not-De-Oet) Il y a plusieurs raisons à cela qui dépassent la portée de cette leçon (Bohr 2007). Une raison majeure est le problème de la dissipation de la chaleur. Essayer de maintenir le même taux de croissance des transistors a nécessité de rendre les transistors plus petits. Mais plus ils deviennent plus petits, plus ils obtiennent lorsqu'ils se réalisent lorsque l'électricité les traverse. La conception des puces modernes vise principalement à essayer de empêcher les choses de fondre!

Cependant, si nous combinons plusieurs CPU à faible puissance, nous pouvons réellement obtenir de meilleures performances que d'un seul processeur haute puissance. Considérez la figure ci-dessous. Avec un seul CPU, il est seulement possible de traiter les données dans un schéma séquentiel ou simultané. Le traitement séquentiel signifie qu'une seule tâche peut être travaillée avant de passer à une autre tâche.

![](./actifs/m2_concurrency.jpg)

*Image de [Kevin Wahome](https://kwahome.medium.com/concurrency-is-not-parélélisme-a5451d1cde8d)*

Dans **un schéma simultané,** Les ordinateurs peuvent basculer de manière transparente entre les tâches si rapidement qu'il semble que plusieurs tâches, ou **threads,** sont en cours de travail simultanément. **concurrence** ou **Multi-threading** est la façon dont les CPU uniques nous ont permis de faire plusieurs tâches pendant les premières décennies de l'ordinateur personnel. **Pour obtenir des ordinateurs et des téléphones mobiles plus rapides aujourd'hui, nous utilisons maintenant plusieurs CPU à faible puissance pour travailler simultanément sur des tâches indépendantes. C'est le schéma parallèle. 

**Aujourd'hui, nous verrons comment plusieurs CPU peuvent être utilisés pour décomposer un problème en parties plus petites qui peuvent être exécutées simultanément.** Certains des outils que nous travaillons facilitent l'utilisation d'un schéma de traitement simultané ou parallèle qu'il peut être difficile de faire la différence entre les deux. Ainsi, dans cette leçon, nous utiliserons les termes «traitement simultané» ou «concurrence» pour faire référence aux schémas de traitement simultanés et parallèles.

---
## Traitement simultané pour les ensembles de données sur le climat

Comme nous l'avons vu précédemment, nous pouvons utiliser `EarthAccess` pour télécharger [Merra-2](https://gmao.gsfc.nasa.gov/reanalysis/Merra-2/) des données de la recherche de la NASA Earthdata. Nous utiliserons les données quotidiennes et agrégées que nous avons utilisées auparavant, avec le `short_name` `"m2sdnxslv"`.

In [ ]:
import earthaccess
import xarray as xr
from matplotlib import pyplot

auth = earthaccess.login()

results = earthaccess.search_data(
    short_name = 'M2SDNXSLV',
    temporal = ("2024-01-01", "2024-05-31"))

#### &#x1f3af; Meilleure pratique

**N'oubliez pas: nous voulons nous assurer que nous ne changez pas accidentellement nos données brutes, donc ces données doivent être téléchargées dans un dossier réservé aux données brutes.**

In [ ]:
# Could take about 1 minute on a broadband connection
earthaccess.download(results, 'data_raw/MERRA2')

Encore une fois, nous utiliserons `xr.open_mfdataset ()` pour ouvrir notre collection de fichiers en tant que `xarray.dataset`.

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4')
ds

Les variables de données Merra-2 qui nous intéressent sont:

- `T2MMAX`, la température quotidienne maximale (degrés C)
- `T2MMEAN`, la température quotidienne moyenne (degrés C)
- `T2MMIN`, la température quotidienne minimale (degrés C)

Notez que nous avons 122 jours de données, de sorte que le cube de données résultant a un axe temporel de 122 pas de temps quotidiens. `xarray` a automatiquement divisé notre ensemble de données en **morceaux de taille égale** qui pourraient être traités indépendamment.

&#x1F449; Dans `xarray`, un **morceau** (également appelé un **bloc**) est un morceau de notre ensemble de données: un sous-ensemble défini le long d'un ou plusieurs axes.

In [ ]:
ds['T2MMEAN']

**La taille et la forme des morceaux sont importantes si nous allons utiliser la concurrence.** Considérons, par exemple, si nous voulions calculer les tendances à long terme. Avec les morceaux que nous avons actuellement, nous n'avons pas pu calculer les tendances car chaque morceau ne contient qu'un seul pas de temps.

Nous pourrions essayer d'utiliser [l'argument `Chunks` de` open_mfdataset () `](https://docs.xarray.dev/en/stable/generated/xarray.open_mfdataset.html) pour spécifier que les Chunks devraient avoir 122 éléments le long des éléments le long du long du long du long du long des éléments le long des éléments le long de la axe `temps`...

In [ ]:
# The "chunks" argument tells xarray what size the chunks should be on one or more axes
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = {'time': 122})
ds['T2MMEAN'].data

Cependant, il est clair que cela n'a pas fonctionné; Chaque morceau n'a toujours qu'un seul pas de temps.

#### &#x1F6A9; <span style = "Color: Red"> Faites attention </DID>

**En effet , il ne peut pas créer de morceaux qui couvrent plusieurs fichiers.** 

Alternativement, nous pouvons dire à `xarray` à quel point chaque morceau devrait être grand le long des axes `lat` et `lon`, car cela ne nécessite pas de compenser plusieurs fichiers. Ci-dessous, nous spécifions des tailles de morceaux qui ne se traduisent en 4 morceaux que pour chaque fichier.

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = {'lat': 182, 'lon': 288})
ds['T2MMEAN'].data

**Si nous avions vraiment besoin de chaque morceau pour contenir l'intégralité de l'axe `time` (122 pas de temps), nous aurions besoin de re-chancher les données *après* la lecture dans tous les fichiers.** Nous pouvons le faire en utilisant [le `chunk ()` méthode d'un `xarray.dataset` `xarray.dataarray`.](https://docs.xarray.dev/en/stable/generated/xarray.dataarray.chunk.html)

In [ ]:
# TODO Re-chunking the data *after* loading is generally inefficient, but might be necessary; 
#    give example of "what if" we were interested in calculating trends

ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4')
ds = ds.chunk({'time': 122})
ds['T2MMEAN'].data

#### &#x1f3af; Meilleure pratique

En général, il est préférable d'utiliser l'argument `chunks` car la réchauffage des données est inefficace. Cependant, dans les cas où vous avez besoin de morceaux pour s'étendre sur plusieurs fichiers, vous devrez re-chantir les données à l'aide de la méthode `chunk ()`.

Dans ce cas, nous n'avons pas besoin de morceaux avec 122 pas de temps. Nous allons bien avec tout ce que fait le groupe `xarray` fait par défaut. Si nous définissons `Chunks = 'Auto'`, alors `xarray` choisira de charger tous les fichiers d'entrée en mémoire à la fois; Par conséquent, il y a un morceau par fichier.

In [ ]:
ds = xr.open_mfdataset('./data_raw/MERRA2/*2024*.nc4', chunks = 'auto')
ds['T2MMEAN'].data

&#x1F449; **Remarquez à quelle vitesse chacun des blocs de code ci-dessus a été exécuté.** C'est parce que `xarray` n'a pas encore chargé de données en mémoire. N'oubliez pas **Évaluation paresseuse?** Encore une fois, `xarray` ne chargera pas de données en mémoire jusqu'à la dernière minute, alors qu'il sera absolument nécessaire de le faire. Et `xarray` n'a pas besoin de charger des données en mémoire afin de nous donner les informations que nous recherchions ci-dessus. Il a simplement lu un peu d'informations de chaque fichier pour apprendre à *représenter* l'ensemble de données complet.

---

## Computing Pet à l'aide de l'équation de Hargreaves

Afin de calculer le rapport de précipitation / pet, nous devrons d'abord utiliser l'équation de Hargreaves pour calculer le TEP:
$$
\text {PET} = 0.0023 \times R_A \times \sqrt {T_ {max} - T_ {min}} \times (T + 17.8)
$$

Ci-dessus, $ R_A $ est le rayonnement solaire haut de gamme (TOA) et $ T $, $ T_ {max} $ et $ T_ {min} $ sont respectivement les températures moyennes, maximales et minimales.

#### &#x1f3af; Meilleure pratique

L'équation de Hargreaves est juste suffisamment complexe pour que nous devons développer plusieurs étapes de traitement des données pour atteindre notre objectif, qui est le rapport précipitation / pépier pour une région définie. Cet effort nécessitera que nous prêtions attention à plusieurs pièges potentiels de la science des données informatiques:

- S'assurer que les étapes de traitement sont effectuées dans le bon ordre, afin que les structures de données et / ou les variables Python soient correctement initialisées.
- S'assurer que les unités de mesure sont correctes et compatibles entre différentes étapes de traitement des données.
- documenter chaque étape de traitement afin que nous puissions identifier les erreurs potentielles et pour qu'un tiers puisse vérifier ou reproduire notre analyse.

Une technique de l'informatique appelée ** Décomposition ** peut nous aider à planifier notre analyse. **Décomposition** implique de briser un problème en une série d'étapes indépendantes et gérables. Nous pourrions décomposer notre problème dans ces étapes:

1. Chargez les entrées de données de température requises.
2. Calculez le rayonnement solaire haut de gamme (TOA).
3. Calculer l'évapotranspiration potentielle (PET) en utilisant l'équation de Hargreaves.
4. Calculez le rapport précipitation / pépier.

**Ces étapes ordonnées devraient nous aider à organiser notre flux de travail d'une manière que quelqu'un d'autre peut facilement comprendre.** Nous avons déjà chargé les données de température requises (étape 1), alors passons au calcul du rayonnement TOA.

### Radiotage informatique (TOA) informatique (TOA)

Voici une fonction de calcul du rayonnement TOA, [basé sur les conseils de la FAO.](Https://www.fao.org/4/x0490e/x0490e07.htm#radiation)

In [ ]:
import numpy as np

def toa_radiation(latitude, doy):
    '''
    Top-of-atmosphere (TOA) radiation for a given latitude (L) and day of year
    (DOY) can be calculated as:

    R = ((24 * 60) / pi) * G * d * (w * sin(L) * sin(D) + cos(L) * cos(D) * sin(w))

    Where G is the solar constant, 0.0820 [MJ m-2 day-1]; d is the (inverse) 
    relative earth-sun distance; w is the sunset hour angle; and D is the solar
    declination angle.
    
    For more information, consult the FAO documentation:

        https://www.fao.org/4/X0490E/x0490e07.htm#radiation
    
    Parameters
    ----------
    latitude : float
        The latitude on earth, in degrees, where southern latitudes
        are represented as negative numbers
    doy : int
        The day of the year (DOY), an integer on [1,366]
    
    Returns
    -------
    Number
        Top-of-atmosphere (TOA) radiation, in [MJ m-2 day-1]
    '''
    assert isinstance(doy, int) or issubclass(doy.dtype.type, np.integer), 'The "doy" argument must be an integer'
    assert np.all(doy >= 1) and np.all(doy <= 366), 'The "doy" argument must be between 1 and 366, inclusive'
    
    solar_constant = 0.0820 # [MJ m-2 day-1]
    pi = 3.14159
    
    # Convert latitude from degrees to radians
    latitude_radians = np.deg2rad(latitude)
    # Inverse Earth-Sun distance (relative), as a function of day-of-year (DOY)
    earth_sun_dist = 1 + 0.0033 * np.cos((doy * 2 * pi) / 365)
    # Solar declination, as a function of DOY
    declination = 0.409 * np.sin(((doy * 2 * pi) / 365) - 1.39)
    
    # Sunset hour angle; we use np.where() below to guard against
    #   warnings where arccos() would return invalid values, which
    #   happens when the argument is outside [-1, 1]
    _hour_angle = -np.tan(latitude_radians) * np.tan(declination)
    _hour_angle = np.where(np.abs(_hour_angle) > 1, np.nan, _hour_angle)
    sunset_hour_angle = np.arccos(_hour_angle)

    # Incident radiation, depends only on the relative earth-sun distance
    inc_radiation = ((24 * 60) / pi) * solar_constant * earth_sun_dist
    return inc_radiation * (sunset_hour_angle * np.sin(latitude_radians) * np.sin(declination) +
            np.cos(latitude_radians) * np.cos(declination) * np.sin(sunset_hour_angle))

### Fonctions bien documentées

**Il y a plusieurs choses à noter sur cette fonction.**

Il existe un **Docstring de fonction** qui fournit des informations riches sur l'objectif et l'utilisation de la fonction. En plus des sections de valeur "Paramètres" et "Retour" importantes, nous avons fourni une forme simple et lisible par l'homme de l'équation que nous utilisons pour calculer le rayonnement TOA. Nous avons également fourni un lien vers le document de la FAO d'où cette équation provient. Ce sont toutes des choses très importantes à inclure afin que quelqu'un d'autre puisse comprendre comment nous calculons le rayonnement TOA. Ces choses nous aident également à vérifier plus tard que nous effectuons correctement les calculs.

Dans la section **Paramètres**, nous nous sommes assurés de définir les unités de mesure requises pour chaque paramètre d'entrée. C'est * extrêmement * important. Dans l'exemple ci-dessus, nous obtiendrions une réponse différente et incorrecte si la «latitude» était donnée en radians au lieu de degrés. Nous avons également indiqué , **le type de données** Python  par exemple, `float`. Ceci est également important à inclure car, lorsqu'un calcul implique le mauvais type de données, il est souvent difficile de comprendre que l'erreur est due à un type de données incorrect.

**Les noms de variables** sont choisis avec soin. Nous utilisons `atitude` au lieu d'un nom comme `X`, ce qui est trop court et pourrait signifier plusieurs choses. Nous avons également défini une variable `latitude_radians` pour distinguer lorsque nous utilisons la latitude dans les radians, par opposition aux degrés. Bien que la «latitude» ait pu être écrite comme `latitude_degrees`, nous avons décidé de compromettre la clarté d'un nom plus court dans ce cas, bien que la clarté soit généralement la plus importante. En fin de compte, il y a des choix subjectifs à faire, mais vous devriez envisager de choisir des noms de variables qui communiquent le sens *et* les unités de mesure de la quantité qu'ils représentent. Si cela est difficile à faire, **les commentaires en ligne** peuvent aider à suivre les unités, comme nous l'avons fait avec le commentaire en ligne à côté de `Solar_Constant`.

**Les constantes** sont définies en haut de notre fonction: `Pi` et `Solar_Constant`. Alors que de nombreuses personnes pourraient reconnaître un nombre comme 3.14 comme le nombre PI, le définissant comme une variable, `Pi`, dans notre fonction le rend plus clair et nous permet de contrôler la précision de ce nombre en un seul endroit. En général, les constantes ne doivent être définies qu'une seule fois!

**Les commentaires** sont utilisés fréquemment. En particulier, lorsqu'il existe des étapes de calcul complexes pour obtenir le `Sunset_Hour_angle`, nous avons un long commentaire au-dessus du code pour expliquer ce qu'il fait. Si nous devons utiliser des variables intermédiaires dans notre calcul, nous pouvons utiliser des noms de variables moins informatifs, comme `_hour_angle`. Dans Python, des noms de variables qui commencent par le soulignement, `_`, signalent aux utilisateurs que la variable est moins importante ou peut être ignorée.

Pour de longs calculs, comme la valeur `retour` de notre fonction, il peut être utile de les diviser en quantités plus petites et plus significatives, en faisant attention à l'ordre des opérations. C'est pourquoi nous avons défini la variable `Inc_radiation`. Lorsqu'un calcul ne peut pas être décomposé en parties significatives, elle peut améliorer la lisibilité à briser l'équation sur plusieurs lignes, comme nous l'avons fait en créant une rupture de ligne après une opération `+`.

Enfin, notez que nous avons inclus **Assertions,** Utilisation du mot-clé `Assert`, pour vous assurer que les utilisateurs appellent correctement cette fonction. Considérez ce qui se passe lorsque le mauvais type de données, ou une valeur hors gamme, est prévu pour l'argument `Doy`:

In [ ]:
toa_radiation(36.1, doy = 14.0)

In [ ]:
toa_radiation(36.1, doy = 500)

#### &#x1F3C1; Défi: rédiger une fonction bien documentée

Maintenant que nous avons examiné ce qui fait une fonction bien documentée, **Écrivez la fonction pour la prochaine étape de notre analyse.** L'équation ci-dessous peut être utilisée pour calculer le TEP. Écrivez une fonction python bien documentée appelée `potentiel_et ()` qui renvoie un animal de compagnie en unités de millimètres par jour (mm jour $ ^ {- 1} $).

$$
\text {PET} = 0.0023 \times R_A \times \sqrt {t_ {max} - t_ {min}} \times (t + 17.8)
$$

Les entrées de la fonction `potentiel_et ()` sont:

- $ r_a $ est le rayonnement solaire haut de l'atmosphère, en mm h $ _2 $ o équivalent par mois
- $ t_ {max} $ est la température maximale mensuelle, en degrés C
- $ t_ {min} $ est la température minimale mensuelle, en degrés C
- $ t $ est la température moyenne mensuelle, en degrés C

**Indice:** Il existe une fonction `np.sqrt ()` pour calculer les racines carrées.

Commencez par modifier la fonction ci-dessous.

In [ ]:
def potential_et(toa_radiation, temp_max, temp_min, temp_mean):
    pass

Développez la cellule ci-dessous pour voir une solution à ce problème.

In [ ]:
def potential_et(toa_radiation, temp_max, temp_min, temp_mean):
    '''
    Calculates potential evapotranspiration, according to the Hargreaves
    equation:

    PET = 0.0023 * R * sqrt(Tmax - Tmin) * (Tmean + 17.8)

    Where R is the top-of-atmosphere (TOA) radiation (mm month-1); Tmax and 
    Tmin are the maximum and minimum monthly air temperatures (degrees C),
    respectively; and Tmean is monthly mean air temperature (degrees C).

    Parameters
    ----------
    toa_radiation : Number
        The top-of-atmosphere (TOA) radiation (mm day-1)
    temp_max : Number
        Maximum monthly air temperature (degrees C)
    temp_min : Number
        Minimum monthly air temperature (degrees C)
    temp_mean : Number
        Average monthly air temperature (degrees C)

    Returns
    -------
    Number
        The potential evapotranspiration (PET) in [mm day-1]
    '''
    return 0.0023 * toa_radiation * np.sqrt(temp_max - temp_min) * (temp_mean + 17.8)

Si la fonction est écrite correctement, lorsqu'elle est appelée avec les arguments ci-dessous, vous devez obtenir une valeur proche de `3.1`.

In [ ]:
potential_et(10, 30, 20, 25)

### Fonctions vectorielles 

Prenons comment appeler les fonctions Python que nous avons écrites.

In [ ]:
toa_radiation(32, 200)

Rappelons que, parce que les tableaux Numpy sont traités comme des numéros, nous pouvons appeler la fonction `toa_radiation()` avec un tableau de nombres pour l'un des arguments.

In [ ]:
lats = np.array([22, 32, 42])

toa_radiation(lats, 200)

En informatique, l'utilisation d'une fonction de cette manière est appelée **vectorisation** et les fonctions compatibles avec à la fois un seul numéro, comme dans `Toa_radiation (32, 200)`, ou un tableau de nombres, comme dans`Toa_radiation ( lats, 200)`, sont appelés **fonctions vectorisées.** Parce que nous travaillons presque toujours avec des tableaux de données, plutôt que des nombres uniques, **les fonctions vectorisées** sont très important.

Par exemple, la vectorisation nous permet de tracer le rayonnement TOA en fonction d'une gamme de dates. La valeur de latitude reste la même pour chaque valeur de `doy`, de sorte que les opérations mathématiques qui dépendent à la fois du jour de l'année et de la latitude ressemblent à l'ajout ou à la multiplication d'un seul nombre (latitude) à un tableau de nombres (jours de la année).

In [ ]:
from matplotlib import pyplot

doy = np.arange(1, 366).astype(np.int32)

rad = toa_radiation(32, doy)
pyplot.plot(doy, rad, 'k-')
pyplot.xlabel('Day of Year')
pyplot.ylabel('TOA Radiation [MJ m-2 day-1]')
pyplot.title('TOA Radiation at 32 deg N latitude')

**Cependant, si nous avons deux réseaux d'entrée ou plus, ils doivent être compatibles.** Ci-dessous, nous obtenons une erreur parce que nous essayons de plusieurs tableaux avec des formes incompatibles. Alors que `Lats` n'a que 3 éléments, `doy` a 365 éléments.

In [ ]:
# Won't work because array shapes are incompatible
toa_radiation(lats, doy)

La seule façon de résoudre ce problème sans changer notre fonction est de rendre nos deux tableaux compatibles. Nous pourrions les remodeler afin qu'ils soient 2 dimensions, $ T \times N $ des tableaux, où $ T $ est le nombre de jours dans `doy` et $ N $ est le nombre de latitudes différentes dans les `lats`. Cela fonctionne bien mais peut être déroutant dans certains cas, car maintenant nous avons un réseau bidimensionnel beaucoup plus grand avec lequel travailler.

In [ ]:
toa_radiation(lats.reshape((1,3)), doy.reshape((365,1)))

### Dériver des variables des coordonnées de `xarray`

La vectorisation sera essentielle pour calculer le TEP pour notre ensemble de données à températures griddées, car nous avons plusieurs latitudes et plusieurs jours de l'année à considérer lors du calcul du rayonnement TOA. Considérez les coordonnées de notre ensemble de données ci-dessous.

In [ ]:
ds.coords

**Nous devons dériver:**

- la latitude pour chaque pixel
- le jour de l'année pour chaque pixel

L'ensemble de données des températures MERRA-2 a 361 bacs de latitude représentés.

In [ ]:
ds.lat.shape

Une étape vers la vectorisation consiste à convertir notre tableau 1 dimension de latitudes en une *grille* de latitudes, car chaque pixel doit avoir une valeur de latitude!

In [ ]:
lats = ds['lat'].values
lats = lats.reshape((361, 1)).repeat(ds['lon'].size, axis = 1)
lats.shape

In [ ]:
pyplot.imshow(lats)
pyplot.colorbar()

#### &#x1F6A9; <span style = "Color: Red"> Faites attention </DID>

Nos latitudes semblent être à l'envers! Faire un intrigue comme celui ci-dessus est un excellent moyen de vérifier notre intitution.

**Cependant, ce n'est pas un problème.** Rappelez-vous que, dans notre `xarray.dataset`, les coordonnées des« lats »passent de -90 (latitude sud) à +90 (latitude nord). Nous voulons que notre grille Latitudes corresponde aux coordonnées de la `xarray.dataset» existante parce que c'est ainsi que les variables de données sont également structurées. Nous pouvons le vérifier en regardant les «valeurs» brutes de l'une de ces variables.

In [ ]:
pyplot.imshow(ds['T2MMIN'].values[0])

**Par conséquent, lorsque nous ajoutons la grille de latitude à notre `xarray.dataseten` tant que nouvelle variable, nous voulons dire `xarray`quels sont les axes correspondants de la grille.** Ci-dessous, nous le faisons en fournissant l'attribution A *Tuple* à la nouvelle variable `"lat_grid"`:

- Le premier élément du tuple est un autre tuple, `('lat', 'lon')` qui spécifie l'ordre et le nom des axes. Ce devraient être des axes qui existent déjà dans l'ensemble de données `ds`.
- Le deuxième élément, `lats`, est notre grille de latitudes.

Ci-dessous, la nouvelle variable `"lat_grid"`est montrée comme ayant des dimensions `(lat, lon)`.

In [ ]:
ds['lat_grid'] = (('lat', 'lon'), lats)
ds

Cependant, les autres variables de données (par exemple, `T2MMIN`, `T2MMAX`) ont des dimensions `(Time, Lat, Lon)`. Il sera beaucoup plus facile de faire des calculs plus tard si `"lat_grid"` a les mêmes dimensions que toutes les autres variables de données.

In [ ]:
lats2 = lats.reshape((1, 361, 576)).repeat(122, axis = 0)
lats2.shape

In [ ]:
ds['lat_grid'] = (('time', 'lat', 'lon'), lats2)
ds

**Maintenant, nous avons la latitude pour chaque pixel. Qu'en est-il du jour de la journée?** Heureusement, cela est facile à dériver de nos coordonnées `time` car ils sont représentés comme le type de données `XArray` `dateTime64[ns]` et ont donc des composants de date comme `'month'` et `dayofyear`. [Vous pouvez en savoir plus sur les composants de date-temps ici.](Https://docs.xarray.dev/en/stable/user-guide/time-series.html#datetime-components)

In [ ]:
ds['time.dayofyear']

---
## En appliquant une fonction à des morceaux indépendants

Maintenant que nous avons les données nécessaires pour calculer le rayonnement TOA, testons notre fonction `toa_radiation()` à l'aide de données à partir d'une seule date.

In [ ]:
test = ds.sel(time = '2024-05-01')

rad = toa_radiation(test['lat_grid'], test['time.dayofyear'])
rad

Notre résultat a une forme de `(1, 361, 576)` parce que notre entrée `"lat_grid"` et `" Time.Dayofyear "" Les tableaux de données ont cette même forme.

Nous pourrions facilement ajouter ce résultat à notre `xarray.Dataset`, `test`, en utilisant la mission:

In [ ]:
test['toa_radiation'] = rad
test

Dans ce cas, la variable `rad` est déjà un `xarray.DataArray`avec les dimensions appropriées. Cependant, c'est une bonne idée de revoir la façon la plus générale d'ajouter une variable de données à un `xarray.dataset`, où nous spécifions les noms et l'ordre des dimensions:

In [ ]:
# NOTE: When rad is already an xarray.DataArray, we need to write rad.data
test['toa_radiation'] = (('time', 'lat', 'lon'), rad.data)

Tractons le résultat pour vérifier si tout a du sens. Nous pouvons voir que le rayonnement TOA est le plus élevé dans l'hémisphère nord, le long d'une bande de latitude d'environ 25 n latitude, ce qui a du sens pour cette période de l'année (mai).

In [ ]:
test['toa_radiation'].plot()

**Comment pouvons-nous appliquer ce que nous venons de faire à l'ensemble de données?** Eh bien, nous n'avions pas besoin de sous-ensemble l'ensemble de données en une seule journée, comme nous l'avons fait ci-dessus. 

&#x1F449; **Cependant, si notre ensemble de données de séries chronologiques était très long, nous voulons peut-être réfléchir à la meilleure façon de profiter de la concurrence.** Le résultat du calcul ne dépend pas des pixels adjacents ou des pas de temps adjacents. Il s'agit d'un type de problème appelé **parallèle de gêne** parce que, en théorie, nous pourrions avoir un nombre indéfiniment grand de tâches exécutées en parallèle sans affecter le résultat.

Avant d'essayer de profiter de la concurrence, passons en revue comment `xarray` est actuellement à la recherche de nos données.

In [ ]:
ds['lat_grid'].chunks

Oups. Notre nouveau jeu de données `"lat_grid"` n'a pas de morceaux. Allons-y et en faisons. L'approche la plus simple serait de laisser chaque fois que le pas (chaque jour) est un morceau différent. Les dimensions `lat` et `lon` peuvent être définies sur `auto` pour indiquer que nous ne nous soucions pas de la façon dont `xarray` les fait.

In [ ]:
# The size of the 'time' axis in each chunk should be 1: a different chunk for each time step
ds['lat_grid'] = ds['lat_grid'].chunk({'lat': 'auto', 'lon': 'auto', 'time': 1})
ds['lat_grid']

### Mappage d'une fonction arbitraire

Nous sommes maintenant prêts à appliquer une fonction à nos morceaux indépendants. Avec **Traitement simultané,** Nous appelons cela **Mapping** une fonction sur les sous-ensembles indépendants de nos données. Dans `xarray`, ces sous-ensembles indépendants sont appelés **morceaux** ou **blocks** (confus,`xarray` utilise les deux termes pour se référer à la même chose). Par conséquent, [la fonction que nous voulons utiliser pour attribuer une tâche à des morceaux indépendants est appelé `xarray.map_blocks ()`.](Https://docs.xarray.dev/en/stable/generated/xarray.map_blocks.html)

Considérez l'exemple ci-dessous, où nous avons une fonction arbitraire appelée «my_function ()». Cette fonction prend un `xarray.dataset` comme son seul argument et renvoie une version modifiée de la variable `"lat_grid"`.

Lorsque nous cartographions `my_function()` sur les blocs indépendants (morceaux), le résultat est clairement un cube de données avec la même forme et les mêmes dimensions.

In [ ]:
# Trivial function that multiplies latitudes by two
def my_function(dataset):
    return dataset['lat_grid'] * 2

xr.map_blocks(my_function, ds)

Encore une fois, notez à quelle vitesse ce code a été exécuté. Rememeber que `xarray` utilise **Évaluation paresseuse** et rien ne s'est encore produit. Nous devons appeler la méthode `calcul()` pour dire que `xarray` nous sommes prêts à charger les données en mémoire et à effectuer notre calcul.

In [ ]:
result = xr.map_blocks(my_function, ds).compute()
result

Considérons maintenant notre fonction `toa_radiation()`. Cette fonction ne sait rien des variables `xarray` ; Il suppose que nous l'appelons avec des numéros ou des tableaux «Numpy». Nous devons donc d'abord créer une fonction qui enveloppe `Toa_radiation()`, comme ci-dessous.

In [ ]:
def toa_radiation_wrapper(dataset):
    return toa_radiation(dataset['lat_grid'], dataset['time.dayofyear'])

Encore une fois, nous devons suivre `map_blocks()` avec `calcul()` si nous sommes prêts à faire le calcul.

In [ ]:
result = xr.map_blocks(toa_radiation_wrapper, ds)

In [ ]:
toa_data = result.compute()

Si nous traçons le même pas qu'avant, nous pouvons voir que nous avons obtenu le même résultat.

In [ ]:
toa_data.sel(time = '2024-05-01').plot()

&#x1F449; Une dernière chose que nous devons faire est une conversion d'unité. $ R_a $ doit être multiplié par 0,408 pour le convertir de [MJ M-2 Day-1] à [MM Day-1]. Allons de l'avant et stockons ce résultat dans notre `xarray.dataset`, appelant la variable `toa_radiation`.

In [ ]:
# Converting TOA Radiation from [MJ m-2 day-1] to [mm H2O day-1]
ds['toa_radiation'] = toa_data * 0.408
ds

#### &#x1f3af; Meilleure pratique

**Nous avons maintenant toutes les données dont nous avons besoin pour calculer TEP, sur une grille globale cohérente, en tant que `xarray.dataset`! Assurez-vous d'inclure des métadonnées au niveau du champ, au cas où nous finirons par partager cet ensemble de données avec d'autres.**

In [ ]:
ds['toa_radiation'].attrs

Il n'y a actuellement aucun attribution. Nous devons au moins en faire un qui clarifie les unités de mesure de notre champ `Toa_radiation`.

In [ ]:
ds['toa_radiation'].attrs['units'] = 'mm H2O day-1'
ds['toa_radiation'].attrs

--- 

## Profilage des ressources de calcul

La combinaison de `xarray` et de `dask` permet de l'utiliser facilement un traitement simultané. Cependant, les ordinateurs sont encore assez rapides, et il peut y avoir plusieurs tâches que nous voulons terminer qui s'exécutent très rapidement sur un seul CPU, même si l'ensemble de données est grand. Le traitement simultané peut être inutile.

#### &#x1f3af; Meilleure pratique

En général, nous devons **profil** les exigences de ressources d'un flux de travail de calcul avant d'implémenter un traitement simultané ou une autre solution. Combien de temps faut-il pour courir? Combien de mémoire nécessite-t-il? Nous devons d'abord répondre à ces questions au lieu de supposer qu'elle prendra trop de temps, ou elle ne rentrera pas dans la mémoire.

Dans ce cas, parce que nous apprenons toujours, nous utilisons un ensemble de données suffisamment petit pour s'intégrer dans la mémoire. Nous le savons parce que, lorsque nous ouvrons l'ensemble de données à l'aide de `open_mfdataset()`, `xarray` nous montrera toujours la quantité de mémoire que chaque `xarray.dataarray` nécessite. Ci-dessous, nous pouvons voir qu'une seule variable Merra-2 ne nécessite que environ 96,77 Mo.

In [ ]:
ds['T2MMEAN'].data

**Notre tâche n'est clairement pas limitée par la taille de la mémoire; Autrement dit, ce n'est pas *lié à la mémoire.*** Mais est-ce lié au processeur? Une façon dont nous pouvons répondre à cette question est de temps combien de temps il faut pour fonctionner sur une petite partie de l'ensemble de données. Par exemple, essayons de courir une seule journée.

In [ ]:
# Note there is exactly one chunk; i.e., the subsequent computation will not use more than one process
first_day = ds.sel(time = '2024-01-01')
first_day['T2MMEAN'].data

Ci-dessous, nous avons réécrit la fonction `potentiel_et()` pour qu'il fonctionne avec un MERRA-2  `xarray.Dataset`.

In [ ]:
def potential_et(dataset):
    '''
    Calculates potential evapotranspiration, according to the Hargreaves
    equation:

    PET = 0.0023 * R * sqrt(Tmax - Tmin) * (Tmean + 17.8)

    Where R is the top-of-atmosphere (TOA) radiation (mm month-1); Tmax and 
    Tmin are the maximum and minimum monthly air temperatures (degrees C),
    respectively; and Tmean is monthly mean air temperature (degrees C).

    Single input argument should be an xarray.Dataset with the following
    data variables:

        T2MMIN: Maximum monthly air temperature (degrees C)
        T2MMAX: Minimum monthly air temperature (degrees C)
        T2MMEAN: Average monthly air temperature (degrees C)
        toa_radiation: The top-of-atmosphere (TOA) radiation (mm day-1)

    Parameters
    ----------
    dataset: xarray.Dataset

    Returns
    -------
    Number
        The potential evapotranspiration (PET) in [mm day-1]
    '''
    return 0.0023 * dataset['toa_radiation'] * np.sqrt(dataset['T2MMAX'] - dataset['T2MMIN']) * (dataset['T2MMEAN'] + 17.8)

### Mesurer le temps du mur d'une tâche avec `timeit`

Nous pouvons utiliser le module `timeit` intégré de Python à temps combien de temps il faut pour exécuter un bloc de code. `timeit` rapportera le **Wall Time** pour notre tâche: le temps qui serait écoulé par une horloge murale pendant que la tâche est en cours d'exécution. Le **Wall Time** est généralement ce qui nous intéresse lorsque nous évaluons combien de temps il faudra pour accomplir une tâche et si cela vaut la peine de diviser cette tâche entre plusieurs processus (simultanés).

& # x1F449; Si nous voulons utiliser `timeit` dans un cahier Jupyter, nous pouvons simplement ajouter la commande magique`%% timeit` en haut du bloc de code Python que nous voulons chronométrer.

In [ ]:
%%timeit

potential_et(first_day)

**Mais attendez!** N'oubliez pas que `xarray` utilise **Évaluation paresseuse.** Nous devons ajouter un appel à `calcul ()` pour nous assurer que nous mesurons réellement le temps de traitement.

In [ ]:
%%timeit

potential_et(first_day).compute()

[Le module `timeIt`](https://docs.python.org/3/library/timeit.html) exécutera en fait la cellule de code plusieurs fois pour obtenir une moyenne du temps qu'il a pris à chaque fois. Le temps nécessaire sera différent pour différentes machines, mais, sur au moins une machine sur laquelle nous avons essayé, il a fallu environ 20 millisecondes (20 ms).

**Si nous voulons savoir combien de temps il faudrait pour exécuter `potentiel_et()` sur l'ensemble de données en utilisant un seul processus,** Nous pouvons faire un ESITMate initial en multipliant le nombre de morceaux (nombre de pas de temps) par le Le temps qu'il a fallu pour exécuter un seul morceau, `First_day`.

In [ ]:
20e-3 * ds.time.size

Donc, cela ne prendrait que 2,4 secondes. Parce que nous travaillons avec un ensemble de données plus petit à des fins d'enseignement, et parce que `potentiel_et ()" est toujours une fonction relativement simple, le traitement de l'ensemble de données sera rapide.

Exécutons `potentiel_et()` sur l'ensemble de l'ensemble de données.

In [ ]:
%%timeit

potential_et(ds).compute()

& # x1F449; ** En fait, il a fallu beaucoup moins de 2,4 secondes pour exécuter `potentiel_et ()` sur l'ensemble de l'ensemble de données. Pourquoi? ** Notre estimation basée sur l'exécution d'un seul morceau sera toujours une surestimation parce que des logiciels comme «Xarray» et l'interprète Python lui-même trouveront souvent des moyens d'optimiser une tâche exécutée dans une boucle, ce qui est essentiellement ce qui se passe Lorsque nous exécutons `potentiel_et ()` sur plusieurs morceaux en utilisant un seul CPU.

Maintenant, comparons l'heure du mur de CPU à un temps mural multiple CPU ci-dessous.

In [ ]:
%%timeit

xr.map_blocks(potential_et, ds).compute()

&#x1F449; **Maintenant, notre tâche s'exécute encore plus lente! Que s'est-il passé?** C'est parce que, lorsque nous utilisons plusieurs CPU ou **processus simultanés,** Il existe une certaine frais générale associée à la planification d'une tâche et à la collecte des résultats. Certaines de ces frais généraux peuvent inclure plusieurs processus (CPU) en attendant leur tour pour lire ou écrire des données, comme un exemple. Lorsque notre tâche, `potentiel_et()`, est déjà assez rapide (comme nous l'avons vu ci-dessus), cette surcharge peut être significative par rapport au temps qu'il faut réellement pour exécuter la tâche.

Les frais généraux multi-processus peuvent également être significatifs lorsqu'il y a un grand nombre de morceaux, car le planificateur a plus de travail à faire pour coordonner la charge de travail pour chaque processus. **La sélection de la bonne taille de morceau est un équilibre** Si les morceaux sont trop grands, il n'y aura pas beaucoup de diminution **du temps de mur** associé à **Traitement simultané** car moins de processus sont utilisés. La mémoire peut également être un problème avec de gros morceaux. Mais si les morceaux sont trop petits, les processus (CPU) attendent souvent de nouvelles données.

[En savoir plus sur la taille et la concurrence du morceau ici.](Https://docs.xarray.dev/en/stable/user-guide/dask.html#chunking-and-performance)

#### &#x1F6A9; <span style = "Color: Red"> Faites attention </DID>

**Lorsque nous utilisons la commande magique `%%TimeIt` dans une cellule de code, il peut interférer avec l'attribution de variables.** En général, nous devons utiliser `%%TimeIt` pour estimer le temps du mur. Si nous voulions obtenir le * résultat * d'un calcul, nous devons l'exécuter dans une cellule de code *sans* la commande magique `%%TimeIt`.

In [ ]:
pet = potential_et(ds).compute()

# If we applied a custom function, the result might inherit its name from
#   an existing input variable; we can reset the name this way
pet.name = 'Potential ET (mm day-1)'
pet.sel(time = '2024-01-01').plot()

En savoir plus sur le module `` Timeit 'ici:

- https://docs.python.org/3/library/timeit.html
- https://sjvrijn.github.io/2019/09/28/how-t-myit.html

## Déduction de la sécheresse hydrologique

Nous sommes maintenant prêts à utiliser notre calcul de PET pour étudier la sécheresse dans Tireret.

In [ ]:
# Subset the global PET data to Tiaret
pet_tiaret = pet.sel(lon = -1.32, lat = 35.37, method = 'nearest')
pet_tiaret

La série temporelle de PET nous raconte comment la perte d'eau (dans l'atmosphère) a changé. Nous pouvons voir que PET a augmenté au cours de cette période, mais ce changement comprend des changements saisonniers attendus à mesure que les changements d'inclinaison de la Terre.

In [ ]:
pet_tiaret.plot()

Afin de calculer notre indice de sécheresse hydrologique (le rapport précipitation / PET), nous devons à nouveau charger les données de précipitation des gazouillis.

In [ ]:
chirps = xr.open_mfdataset('data_raw/CHIRPS/CHIRPS-v2_Africa_monthly_2014-2024.nc')

# Subset the CHIRPS data to Tiaret
chirps_tiaret = chirps['precip'].sel(x = slice(0.8, 1.8), y = slice(36.1, 35.1))
chirps_tiaret

Nous voulons combiner ces deux ensembles de données ensemble mais:

- Les données sur les précipitations de gazouillis sont mensuelles (mm mois $ ^ {- 1} $)
- Les données TEP que nous avons calculées à partir de Merra-2 sont quotidiennes (MM Day $ ^ {- 1} $)

Comment pouvons-nous résoudre cette différence de résolution temporelle? Nous pourrions résumer les données TEP aux pas de temps mensuels, mais il serait préférable que nous puissions préserver sa résolution temporelle plus fine. 

Au lieu de cela, rééchantillonnons les données des précipitations de gazouillis aux pas de temps quotidiens. ** Nous supposerons que les précipitations mensuelles totales sont réparties uniformément chaque mois. ** Nous pouvons y parvenir facilement en utilisant la méthode `` la plus la plus proche () ', qui effectue une interpolation de plus proche de l'Eightbor, en copie essentiellement la valeur mensuelle à chaque jour.

In [ ]:
# Increasing the frequency of our monthly dataset to daily using nearest-neighbor interpolation
chirps_tiaret_resampled = chirps_tiaret.isel(time = slice(120, 125)).resample(time = 'D').nearest()
chirps_tiaret_resampled

**Nous ne devons pas oublier de diviser le résultat par 30 (jours) pour obtenir les précipitations quotidiennes moyennes.** Il s'agit d'une approximation simple car certains mois n'ont pas exactement 30 jours.

In [ ]:
chirps_tiaret_daily = chirps_tiaret_resampled.mean(['x', 'y']) / 30
chirps_tiaret_daily

Nous sommes maintenant prêts à calculer le rapport Précipitation / PET pour Tireret.

In [ ]:
ratio = 100 * chirps_tiaret_daily.values / pet_tiaret.values

pyplot.figure(figsize = (12, 4))

# Use the time coordinates as the X-axis values
pyplot.plot(pet['time'].values, ratio, 'k-')

En soi, le graphique ci-dessus ne nous dit pas à quel point la sécheresse dans Tireret est grave. Bien que les précipitations dans la région aient reconstitué moins de 5% de son eau perdue au cours des derniers mois, cela pourrait faire partie du cycle saisonnier normal. En fait, nous savons que janvier à avril est une période relativement humide pour Tireret, mais la question demeure: ** Pouvons-nous comparer cette année aux dernières années? **

---

## Résumé

- Lorsque nous lisons dans un ensemble de données en utilisant `xarray.open_mfdataset()`, qu'il s'agisse de plusieurs fichiers ou non, le `xarray.dataset` est une représentation de l'ensemble de données entier: elle n'est pas encore chargée en mémoire. L'ensemble de données est également automatiquement sous-ensemble en **morceaux,** ou plus petits données, en fonction des ressources de notre ordinateur. Ces morceaux pourraient être traités indépendamment en utilisant **un traitement simultané,** où une seule tâche est effectuée séparément sur chaque morceau, en utilisant un ou plusieurs processeurs.

- Les ressources informatiques incluent la mémoire d'un ordinateur, la vitesse d'horloge du CPU (à quelle vitesse il peut terminer une tâche) et la vitesse de lecture du système de fichiers. Les tâches qui sont principalement limitées par chacune de ces ressources sont appelées **Tasques liées à la mémoire, liées au CPU,** et **Entrée-sortie (IO),** respectivement.

- **Les fonctions Vectrized** sont des fonctions qui sont écrites pour fonctionner avec des entrées qui pourraient être un numéro unique ou un tableau de nombres, comme un `numpy.ndarray` ou` xarray.dataarray`. Cela aide à augmenter une analyse des ensembles de données arbitrairement importants. 

- Lorsque vous utilisez **le traitement simultané,** La taille du morceau doit être sélectionnée avec soin. Trop de petits morceaux peuvent entraîner beaucoup de frais généraux. Avec trop de frais généraux, une tâche peut fonctionner plus lentement avec plusieurs processus qu'avec un seul processus.

- Une fonction personnalisée peut être appliquée à plusieurs morceaux indépendants, également appelés **blocs,** en utilisant `xarray.map_blocks ()`.

---

### Plus de ressources

- Le National Center for Atmospheric Research (NCAR) a un excellent article sur ["Utilisation de` Dask` pour mettre à l'échelle votre analyse des données. "](Https://ncar.github.io/xarray-dask-esds-2024/notebooks/02-dask-intro.html)
- [Computation parallèle avec `dask`](docs.xarray.dev/en/stable/user-guide/dask.html)
- [Tutoriel de Sander Van Rijn sur l'utilisation du module `TimeIt`.](Https://sjvrijn.github.io/2019/09/28/how-to-timeit.html)

### références

Bohr, Mark. 2007. ["Une rétrospective de 30 ans sur le papier de mise à l'échelle MOSFET de Dennard." [https://www.eng.auburn.edu/~agrawvd/course/reading/lowp/boh07.pdf diplomatique](https://www.eng.auburn.edu/~agrawvd/course/reading/lowp/boh07.pdf)

Moore, Gordon E. 1965. "Embrassant plus de composants sur des circuits intégrés" *Electronics Magazine.*